In [1]:
import gradio as gr
import random
import numpy as np
from dotenv import load_dotenv, find_dotenv
import os
import json
import chromadb
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader,
    StorageContext,
    Settings
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.llms.gemini import Gemini
from llama_index.core.schema import QueryBundle
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

_ = load_dotenv(find_dotenv())

# Initialize Gemini
os.environ["GEMINI_API_KEY"] = os.getenv('GEMINI_API_KEY')
llm = Gemini(model="models/gemini-2.0-flash", temperature=1)

# Initialize embedding model
lc_embed_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-small"
)
embed_model = LangchainEmbedding(lc_embed_model)
Settings.embed_model = embed_model

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chromadb")

In [9]:
chroma_client = chromadb.PersistentClient(path="../chromadb")

def load_database():
    """
    Connect database with ChromaDB backend.
    :return: LlamaIndex index
    """
    # Setup ChromaDB
    chroma_collection = chroma_client.get_collection("unit1_db")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create index with custom embedding model
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model,
    )
    
    return index


In [10]:
load_database()